In [1]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
parser = argparse.ArgumentParser(description='PyTorch actor-critic example')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G', help='discount factor (default: 0.99)')
parser.add_argument('--seed', type=int, default=543, metavar='N', help='random seed (default: 543)')
# parser.add_argument('--render', action='store_true', help='render the environment')
parser.add_argument('--render', type=bool,default=True, help='render the environment')
parser.add_argument('--log-interval', type=int, default=10, metavar='N', help='interval between training status logs (default: 10)')
parser.add_argument('-f','--file',help='Path for input file. (Dummy arg to enable execution in notebook.)' )
args = parser.parse_args() 

In [3]:
env = gym.make('CartPole-v0')
env.seed(args.seed)
torch.manual_seed(args.seed)

In [4]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = F.softmax(self.action_head(x), dim=-1)
        state_values = self.value_head(x)
        return action_scores, state_values

model = Policy()
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
optimizer = optim.Adam(model.parameters(), lr=3e-2)

#Get the smallest possible non-zero number on this machine
eps = np.finfo(np.float32).eps.item()

def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()

def finish_episode():
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    returns = []
    for r in model.rewards[::-1]:
        R = r + args.gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()
        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    loss.backward()
    optimizer.step()
    del model.rewards[:]
    del model.saved_actions[:]

def main():
    running_reward = 10
    for i_episode in count(1):
        state, ep_reward = env.reset(), 0
        for t in range(1, 10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if args.render:
                env.render()
            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
        if i_episode % args.log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break
main()

Episode 10	Last reward: 9.00	Average reward: 10.67
Episode 20	Last reward: 10.00	Average reward: 14.35
Episode 30	Last reward: 11.00	Average reward: 12.54
Episode 40	Last reward: 10.00	Average reward: 11.31
Episode 50	Last reward: 12.00	Average reward: 10.79
Episode 60	Last reward: 9.00	Average reward: 10.43
Episode 70	Last reward: 9.00	Average reward: 10.51
Episode 80	Last reward: 12.00	Average reward: 11.04
Episode 90	Last reward: 10.00	Average reward: 10.78
Episode 100	Last reward: 32.00	Average reward: 12.34
Episode 110	Last reward: 12.00	Average reward: 13.97
Episode 120	Last reward: 26.00	Average reward: 19.78
Episode 130	Last reward: 38.00	Average reward: 28.38
Episode 140	Last reward: 38.00	Average reward: 42.71
Episode 150	Last reward: 24.00	Average reward: 34.16
Episode 160	Last reward: 11.00	Average reward: 28.39
Episode 170	Last reward: 39.00	Average reward: 27.95
Episode 180	Last reward: 30.00	Average reward: 46.19
Episode 190	Last reward: 56.00	Average reward: 49.18
Episo